# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [24]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [27]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "o3"
openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3:8b"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

#check if ollama is running
def check_ollama():
    try:
        response = openai.chat.completions.create(
            model=MODEL,
            messages=[{"role": "system", "content": "You are a helpful assistant."}]
        )
        return "Model is running: " + response.choices[0].message.content
    except Exception as e:
        print(f"Ollama is not running: {e}")
        return False
check_ollama()


OpenAI API Key exists and begins sk-proj-
Ollama is not running: Error code: 404 - {'error': {'message': 'Your organization must be verified to use the model `o3`. Please go to: https://platform.openai.com/settings/organization/general and click on Verify Organization. If you just verified, it can take up to 15 minutes for access to propagate.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}


False

In [3]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."
system_message += "If the user asks for a flight, ask them for the time that they want to fly, and the destination. And show the user the flight options."
system_message+="If the user want to book a flight, ask them for their name, and then book the flight for them. "

In [4]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [5]:
ticket_prices = {
    "london": "$799",
    "paris": "$899",
    "tokyo": "$1400",
    "berlin": "$499",
    "madrid": "$600",
    "new york": "$1200",
    "san francisco": "$1500"
}

available_cities_times = {
    "london": ["08:00", "12:00", "16:00"],
    "paris": ["09:00", "13:00", "17:00"],
    "tokyo": ["10:00", "14:00", "18:00"],
    "berlin": ["11:00", "15:00", "19:00"],
    "madrid": ["12:00", "16:00", "20:00"],
    "new york": ["13:00", "17:00", "21:00"],
    "san francisco": ["14:00", "18:00", "22:00"]
}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

def get_available_times(destination_city):
    print(f"Tool get_available_times called for {destination_city}")
    city = destination_city.lower()
    return available_cities_times.get(city, "Unknown")

def book_ticket(destination_city, time):
    print(f"Tool book_ticket called for {destination_city} at {time}")
    city = destination_city.lower()
    # Check if city exists
    if city not in ticket_prices:
        return "City not found."
    # Check if time is available for the city
    if time not in available_cities_times.get(city, []):
        return "Time not available."
    # Simulate a successful booking by removing the time
    available_cities_times[city].remove(time)
    print(f"Booking confirmed for {city} at {time}. Price: {ticket_prices[city]}")
    return f"Ticket booked to {city.title()} at {time}. Price: {ticket_prices[city]}"

In [6]:
# get_ticket_price("San Francisco")  # Example call to the function
# get_available_times("Paris")  # Example call to the function
# book_ticket("Tokyo", "10:00")  # Example call to the function
# #check if we can remove the time from available times
# book_ticket("Tokyo", "10:00")  # Example call to the function again to see if the time is removed
# book_ticket("Tokyo", "11:00")  # Example call to the function with an unavailable time
# book_ticket("Unknown City", "10:00")  # Example call to the function with an unknown city
# #print the available times for Tokyo to see if the time was removed
# print(available_cities_times["tokyo"])  # Should show remaining times for Tokyo

In [7]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}
avaiability_function = {
    "name": "get_available_times",
    "description": "Get the available flight times to the destination city. Call this whenever you need to know the available flight times, for example when a customer asks 'What time is the next flight to this city?'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}
book_function = {
    "name": "book_ticket",
    "description": "Book a ticket to the destination city at the specified time. Call this whenever a customer wants to book a ticket, for example when they say 'Book a ticket to this city at this time'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
            "time": {
                "type": "string",
                "description": "The time of the flight that the customer wants to book",
            },
        },
        "required": ["destination_city", "time"],
        "additionalProperties": False
    }
}


In [8]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}, {"type": "function", "function": avaiability_function}, {"type": "function", "function": book_function}]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [9]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [18]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    available_times = get_available_times(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price, "available_times": available_times, "book_function": book_function}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [23]:
#check the avaiable times for tokyo
print(get_available_times("London")) 

Tool get_available_times called for London
['08:00', '12:00', '16:00']


Tool get_ticket_price called for London
Tool get_available_times called for London


In [20]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for London
Tool get_available_times called for London
Tool get_ticket_price called for London
Tool get_available_times called for London
